In [1]:
import os
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import re

In [4]:
def check_if_character(line, punct_set):
    if "(" in line:
        line=line[:line.index("(")].strip()
    if "{" in line:
        line=line[:line.index("{")].strip()
    if '[' in line:
        line=line[:line.index('[')].strip()
    if "/" in line:
            line=line[:line.index('/')-1].strip()
    if line.upper()==line and line.isupper() and line[-1] not in punct_set and line.count(' ')<4:
        
        return True, line
    else:
        return False, ""


In [61]:
##for when dictionary is available
##for movie in total_dictionary
##if new_character not in unacceptable starters and new_character in movie['characters]
##if movie['is_villain']==new_character:
##   current_character_dict['is_villain']=True


unacceptable_starters=["VOICE (cont'd)", "VOICE (CONT'D)", "VOICE OVER (CONT'D)", "VOICE OVER (cont'd)", "DISSOLVE", "CUT", "CUT TO", 'FADE', 'FADE OUT', 'FADE IN', 'PAN', 'CONTINUED', "CONT'D", '', ' ', "VOICE", "VOICE OVER", 'CUT TO', 'DISSOLVE TO', 'THE END', 'FADE TO BLACK', "DISSOLVE TO:", "CUT TO:", "FADE TO:"]

test_set=["blackdahliathe_dialog.txt", "basicinstinct_dialog.txt", "basic_dialog.txt","girlwiththedragontattoothe_dialog.txt", "manhattanmurdermystery_dialog.txt", 'afewgoodmen_dialog.txt', "8mm_dialog.txt", "backdraft_dialog.txt"]

movie_character_dicts=[]


punct_remover=str.maketrans('','', '"#$%&()*+-/:;<=>?@[\\]^_`{|}~')

for movie in test_set:
    path=os.path.join("scripts", "samples", movie)

    with open(path, "r", encoding="utf-8") as f:
        movie_script=f.readlines()
        start_of_content=False

        this_movie_dicts=[]
        current_character_dict={}
        movie_title=""
        current_character_name="" 
        # current_character_dialogue=[]
        year=0      
        
        for line in movie_script:
            line=line.strip()

            #checks for the title and the first character, otherwise skips the header stuff
            if not start_of_content:
                line=line.strip('"')

                if movie_title=="" and line.translate(punct_remover)==line and line not in unacceptable_starters:
                    movie_title=line

                if year==0 and len(re.findall('\d{4,}', line))>0:
                    year=re.findall('\d{4,}',line)[0]

                is_character, new_character_name = check_if_character(line, string.punctuation)

                if is_character and (new_character_name not in unacceptable_starters):
                    start_of_content=True
                    current_character_name=new_character_name
                    current_character_dict={"character": current_character_name, "movie_title": movie_title, "year":year, "is_villain": False, "raw_dialogue":""}
                    this_movie_dicts.append(current_character_dict)
            
            #if in the middle of the script and you have a current character, check if this line is a new character otherwise add the dialogue to this character's list
            else:
                is_new_character, new_character_name = check_if_character(line, string.punctuation)
                if is_new_character:
                    current_character_name=new_character_name
                    if current_character_name not in unacceptable_starters:
                        if current_character_name not in [entry['character'] for entry in this_movie_dicts]:
                            current_character_dict={"character": current_character_name, "movie_title": movie_title, "year":year, "is_villain": False, "raw_dialogue":""}
                            this_movie_dicts.append(current_character_dict)
                        else:
                            current_character_dict=[entry for entry in this_movie_dicts if entry['character']==current_character_name][0]
                        
                else:
                    if current_character_name in unacceptable_starters:
                        pass
                    else:
                        if len(line)>0 and line[0]!='(' and line not in unacceptable_starters:
                            current_character_dict['raw_dialogue']=current_character_dict['raw_dialogue']+' ' + line.translate(punct_remover).strip()

        movie_character_dicts+=this_movie_dicts
movie_character_dicts=[entry for entry in movie_character_dicts if len(entry['raw_dialogue'])!=0]
for entry in movie_character_dicts:
    entry['num_words']=entry['raw_dialogue'].count(' ')+1                            
            






In [62]:
set([entry['movie_title'] for entry in movie_character_dicts])

{'8MM',
 'A FEW GOOD MEN',
 'BACKDRAFT',
 'BASIC INSTINCT',
 'Basic',
 'MANHATTAN MURDER MYSTERY',
 'THE GIRL WITH THE DRAGON TATTOO',
 'The Black Dahlia'}

In [63]:
#fixes where characters are referred to by separate names
def combine_characters(movie_character_dict, movie_title, character_name_to_keep, character_name_to_remove):
    movie_subset=[entry for entry in movie_character_dict if entry['movie_title']==movie_title]
    if character_name_to_remove not in [entry['character'] for entry in movie_subset] or character_name_to_keep not in [entry['character'] for entry in movie_subset]:
        return
    to_remove=[entry for entry in movie_character_dict if entry['character']==character_name_to_remove][0]
    keeper=[entry for entry in movie_subset if entry['character']==character_name_to_keep][0]
    keeper['raw_dialogue']+=to_remove['raw_dialogue']
    keeper['num_words']+=to_remove['num_words']
    movie_character_dict.remove(to_remove)

In [64]:
#random fixes

combine_characters(movie_character_dicts, "8MM", "AMY", "AMY'S VOICE")
combine_characters(movie_character_dicts, "8mm", "DINO VELVET", "DINO")
combine_characters(movie_character_dicts, "8mm", "DINO VELVET", "DINO VELVET VOICE")
combine_characters(movie_character_dicts, "8mm", "WELLES", "WELLES VOICE")
combine_characters(movie_character_dicts, "8mm", "WELLES", "WELLES' VOICE")

combine_characters(movie_character_dicts, "MANHATTAN MURDER MYSTERIES", "HELEN", "HELEN'S VOICE")

combine_characters(movie_character_dicts, 'The Black Dahlia', "CAPTAIN VASQUEZ", 'VASQUEZ')
combine_characters(movie_character_dicts, 'The Black Dahlia', "JOHNNY VOGEL", 'JOHNNY')
combine_characters(movie_character_dicts, 'The Black Dahlia', "JOHNNY VOGEL", 'VOGEL')
combine_characters(movie_character_dicts, "The Black Dahlia", "LEE BLANCHARD", "LEE")

combine_characters(movie_character_dicts, 'The Black Dahlia', "ELLIS LOEW", 'LOEW')
combine_characters(movie_character_dicts, 'The Black Dahlia', "RUSS MILLARD", 'MILLARD')
combine_characters(movie_character_dicts, 'BASIC INSTINCT', "CAPTAIN TALCOTT", 'TALCOTT')
combine_characters(movie_character_dicts, 'BASIC INSTINCT', "CAPTAIN TALCOTT", 'CAPT. TALCOTT')

combine_characters(movie_character_dicts, 'Basic', "DUNBAR", 'DUN BAR')
combine_characters(movie_character_dicts, 'Basic', "MUELLER", 'MUE:LLER')
combine_characters(movie_character_dicts, 'Basic', "OSBORNE", 'OSB0RNE')

combine_characters(movie_character_dicts, 'THE GIRL WITH THE DRAGON TATTOO', "GREGOR", 'GREGER')
combine_characters(movie_character_dicts, 'THE GIRL WITH THE DRAGON TATTOO', "GREGOR", 'GREGER')
combine_characters(movie_character_dicts, 'THE GIRL WITH THE DRAGON TATTOO', "BLOMKVIST", 'BLOMVIST')
combine_characters(movie_character_dicts, 'THE GIRL WITH THE DRAGON TATTOO', "HARRIET", 'HARRIE')
combine_characters(movie_character_dicts, 'THE GIRL WITH THE DRAGON TATTOO', "WENNERSTROM", 'WENNERSTROM ON TV')
combine_characters(movie_character_dicts, 'THE GIRL WITH THE DRAGON TATTOO', "VANGER", 'YOUNGER VANGER')

In [65]:
sample_dataframe=pd.DataFrame.from_dict(movie_character_dicts)

In [66]:
sample_dataframe.to_csv("sample.csv")